In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz


In [2]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

In [3]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [4]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [5]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    print('conv1', conv1.shape)
    conv1 = maxpool2d(conv1, k=2)
    print('conv1 after maxpool', conv1.shape)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    print('conv2', conv2.shape)
    conv2 = maxpool2d(conv2, k=2)
    print('conv2 after max pool', conv2.shape)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    print('fc1 after reshape', fc1.shape)
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    print('fc1 after wX+b', fc1.shape)
    fc1 = tf.nn.relu(fc1)
    print('fc1 after relu', fc1.shape)
    fc1 = tf.nn.dropout(fc1, dropout)
    print('fc1 after dropout', fc1.shape)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    print('out', out.shape)
    return out

In [6]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

conv1 (?, 28, 28, 32)
conv1 after maxpool (?, 14, 14, 32)
conv2 (?, 14, 14, 64)
conv2 after max pool (?, 7, 7, 64)
fc1 after reshape (?, 3136)
fc1 after wX+b (?, 1024)
fc1 after relu (?, 1024)
fc1 after dropout (?, 1024)
out (?, 10)
Epoch  1, Batch   1 -Loss: 59872.5312 Validation Accuracy: 0.160156
Epoch  1, Batch   2 -Loss: 37092.7266 Validation Accuracy: 0.179688
Epoch  1, Batch   3 -Loss: 33908.7500 Validation Accuracy: 0.164062
Epoch  1, Batch   4 -Loss: 30478.5703 Validation Accuracy: 0.160156
Epoch  1, Batch   5 -Loss: 32879.7031 Validation Accuracy: 0.160156
Epoch  1, Batch   6 -Loss: 30955.8711 Validation Accuracy: 0.183594
Epoch  1, Batch   7 -Loss: 23883.5859 Validation Accuracy: 0.203125
Epoch  1, Batch   8 -Loss: 25327.6719 Validation Accuracy: 0.203125
Epoch  1, Batch   9 -Loss: 22573.6758 Validation Accuracy: 0.195312
Epoch  1, Batch  10 -Loss: 22694.9453 Validation Accuracy: 0.191406
Epoch  1, Batch  11 -Loss: 18888.8359 Validation Accuracy: 0.191406
Epoch  1, Batch  12

Epoch  1, Batch 119 -Loss:  3268.2856 Validation Accuracy: 0.675781
Epoch  1, Batch 120 -Loss:  3140.9526 Validation Accuracy: 0.675781
Epoch  1, Batch 121 -Loss:  3796.1997 Validation Accuracy: 0.683594
Epoch  1, Batch 122 -Loss:  3804.9995 Validation Accuracy: 0.691406
Epoch  1, Batch 123 -Loss:  3986.8811 Validation Accuracy: 0.699219
Epoch  1, Batch 124 -Loss:  4416.4316 Validation Accuracy: 0.699219
Epoch  1, Batch 125 -Loss:  4549.6650 Validation Accuracy: 0.687500
Epoch  1, Batch 126 -Loss:  3900.9438 Validation Accuracy: 0.683594
Epoch  1, Batch 127 -Loss:  4391.0352 Validation Accuracy: 0.691406
Epoch  1, Batch 128 -Loss:  4014.8843 Validation Accuracy: 0.695312
Epoch  1, Batch 129 -Loss:  3703.9336 Validation Accuracy: 0.699219
Epoch  1, Batch 130 -Loss:  3569.8999 Validation Accuracy: 0.691406
Epoch  1, Batch 131 -Loss:  3872.3506 Validation Accuracy: 0.683594
Epoch  1, Batch 132 -Loss:  4333.5317 Validation Accuracy: 0.699219
Epoch  1, Batch 133 -Loss:  3752.7163 Validation

Epoch  1, Batch 240 -Loss:  2367.7437 Validation Accuracy: 0.789062
Epoch  1, Batch 241 -Loss:  2390.8335 Validation Accuracy: 0.789062
Epoch  1, Batch 242 -Loss:  2739.3984 Validation Accuracy: 0.792969
Epoch  1, Batch 243 -Loss:  1990.1997 Validation Accuracy: 0.804688
Epoch  1, Batch 244 -Loss:  2568.4077 Validation Accuracy: 0.792969
Epoch  1, Batch 245 -Loss:  1310.1748 Validation Accuracy: 0.796875
Epoch  1, Batch 246 -Loss:  2395.2041 Validation Accuracy: 0.792969
Epoch  1, Batch 247 -Loss:  1936.8346 Validation Accuracy: 0.800781
Epoch  1, Batch 248 -Loss:  1717.9474 Validation Accuracy: 0.792969
Epoch  1, Batch 249 -Loss:  1721.5657 Validation Accuracy: 0.789062
Epoch  1, Batch 250 -Loss:  1786.5669 Validation Accuracy: 0.789062
Epoch  1, Batch 251 -Loss:  2299.4414 Validation Accuracy: 0.792969
Epoch  1, Batch 252 -Loss:  2131.9819 Validation Accuracy: 0.792969
Epoch  1, Batch 253 -Loss:  1995.8688 Validation Accuracy: 0.792969
Epoch  1, Batch 254 -Loss:  2755.9067 Validation

Epoch  1, Batch 361 -Loss:  1575.5408 Validation Accuracy: 0.812500
Epoch  1, Batch 362 -Loss:  1417.6962 Validation Accuracy: 0.812500
Epoch  1, Batch 363 -Loss:  1783.9165 Validation Accuracy: 0.816406
Epoch  1, Batch 364 -Loss:  1562.9227 Validation Accuracy: 0.816406
Epoch  1, Batch 365 -Loss:  1432.1654 Validation Accuracy: 0.820312
Epoch  1, Batch 366 -Loss:  1590.7158 Validation Accuracy: 0.820312
Epoch  1, Batch 367 -Loss:  1131.8921 Validation Accuracy: 0.820312
Epoch  1, Batch 368 -Loss:  1831.7893 Validation Accuracy: 0.808594
Epoch  1, Batch 369 -Loss:  1572.2457 Validation Accuracy: 0.804688
Epoch  1, Batch 370 -Loss:  1457.0139 Validation Accuracy: 0.808594
Epoch  1, Batch 371 -Loss:  1656.7871 Validation Accuracy: 0.808594
Epoch  1, Batch 372 -Loss:  1657.3242 Validation Accuracy: 0.804688
Epoch  1, Batch 373 -Loss:  1823.0400 Validation Accuracy: 0.816406
Epoch  1, Batch 374 -Loss:  1097.5559 Validation Accuracy: 0.812500
Epoch  1, Batch 375 -Loss:  1109.1152 Validation

Epoch  2, Batch  53 -Loss:  1465.0699 Validation Accuracy: 0.832031
Epoch  2, Batch  54 -Loss:   966.0222 Validation Accuracy: 0.832031
Epoch  2, Batch  55 -Loss:  1368.6252 Validation Accuracy: 0.832031
Epoch  2, Batch  56 -Loss:   762.3005 Validation Accuracy: 0.824219
Epoch  2, Batch  57 -Loss:  1793.1436 Validation Accuracy: 0.820312
Epoch  2, Batch  58 -Loss:  1140.4976 Validation Accuracy: 0.824219
Epoch  2, Batch  59 -Loss:  1399.7961 Validation Accuracy: 0.820312
Epoch  2, Batch  60 -Loss:  1337.0175 Validation Accuracy: 0.816406
Epoch  2, Batch  61 -Loss:  1719.2678 Validation Accuracy: 0.820312
Epoch  2, Batch  62 -Loss:  1304.1558 Validation Accuracy: 0.816406
Epoch  2, Batch  63 -Loss:  1448.1221 Validation Accuracy: 0.816406
Epoch  2, Batch  64 -Loss:  1280.4473 Validation Accuracy: 0.820312
Epoch  2, Batch  65 -Loss:  1220.8965 Validation Accuracy: 0.812500
Epoch  2, Batch  66 -Loss:  1011.4042 Validation Accuracy: 0.816406
Epoch  2, Batch  67 -Loss:  1663.2711 Validation

...

Epoch  9, Batch 317 -Loss:   321.8920 Validation Accuracy: 0.894531
Epoch  9, Batch 318 -Loss:   490.6913 Validation Accuracy: 0.898438
Epoch  9, Batch 319 -Loss:   388.6413 Validation Accuracy: 0.898438
Epoch  9, Batch 320 -Loss:   345.9279 Validation Accuracy: 0.894531
Epoch  9, Batch 321 -Loss:   277.6290 Validation Accuracy: 0.894531
Epoch  9, Batch 322 -Loss:   308.0508 Validation Accuracy: 0.894531
Epoch  9, Batch 323 -Loss:   413.3169 Validation Accuracy: 0.894531
Epoch  9, Batch 324 -Loss:   380.2045 Validation Accuracy: 0.890625
Epoch  9, Batch 325 -Loss:   199.8306 Validation Accuracy: 0.894531
Epoch  9, Batch 326 -Loss:   225.1360 Validation Accuracy: 0.890625
Epoch  9, Batch 327 -Loss:   324.1469 Validation Accuracy: 0.894531
Epoch  9, Batch 328 -Loss:   492.7473 Validation Accuracy: 0.894531
Epoch  9, Batch 329 -Loss:   309.1364 Validation Accuracy: 0.898438
Epoch  9, Batch 330 -Loss:   375.3887 Validation Accuracy: 0.898438
Epoch  9, Batch 331 -Loss:   319.1830 Validation

Epoch 10, Batch   9 -Loss:   216.6613 Validation Accuracy: 0.898438
Epoch 10, Batch  10 -Loss:   338.4898 Validation Accuracy: 0.898438
Epoch 10, Batch  11 -Loss:   251.3618 Validation Accuracy: 0.898438
Epoch 10, Batch  12 -Loss:   349.1950 Validation Accuracy: 0.898438
Epoch 10, Batch  13 -Loss:   366.3883 Validation Accuracy: 0.902344
Epoch 10, Batch  14 -Loss:   260.0479 Validation Accuracy: 0.902344
Epoch 10, Batch  15 -Loss:   235.1181 Validation Accuracy: 0.894531
Epoch 10, Batch  16 -Loss:   386.5135 Validation Accuracy: 0.894531
Epoch 10, Batch  17 -Loss:   247.3739 Validation Accuracy: 0.902344
Epoch 10, Batch  18 -Loss:   311.8920 Validation Accuracy: 0.898438
Epoch 10, Batch  19 -Loss:   238.7119 Validation Accuracy: 0.902344
Epoch 10, Batch  20 -Loss:   324.5211 Validation Accuracy: 0.902344
Epoch 10, Batch  21 -Loss:   330.9018 Validation Accuracy: 0.898438
Epoch 10, Batch  22 -Loss:   532.0901 Validation Accuracy: 0.906250
Epoch 10, Batch  23 -Loss:   166.8810 Validation

Epoch 10, Batch 130 -Loss:   104.0927 Validation Accuracy: 0.894531
Epoch 10, Batch 131 -Loss:   434.9372 Validation Accuracy: 0.894531
Epoch 10, Batch 132 -Loss:   281.8107 Validation Accuracy: 0.894531
Epoch 10, Batch 133 -Loss:   343.7782 Validation Accuracy: 0.894531
Epoch 10, Batch 134 -Loss:   281.4574 Validation Accuracy: 0.894531
Epoch 10, Batch 135 -Loss:   396.0729 Validation Accuracy: 0.894531
Epoch 10, Batch 136 -Loss:   489.7437 Validation Accuracy: 0.898438
Epoch 10, Batch 137 -Loss:   285.0700 Validation Accuracy: 0.894531
Epoch 10, Batch 138 -Loss:   210.6901 Validation Accuracy: 0.894531
Epoch 10, Batch 139 -Loss:   194.6355 Validation Accuracy: 0.894531
Epoch 10, Batch 140 -Loss:   333.0495 Validation Accuracy: 0.894531
Epoch 10, Batch 141 -Loss:   369.3255 Validation Accuracy: 0.894531
Epoch 10, Batch 142 -Loss:   366.3337 Validation Accuracy: 0.898438
Epoch 10, Batch 143 -Loss:   347.9246 Validation Accuracy: 0.898438
Epoch 10, Batch 144 -Loss:   127.5862 Validation

Epoch 10, Batch 251 -Loss:   191.4596 Validation Accuracy: 0.886719
Epoch 10, Batch 252 -Loss:   229.1606 Validation Accuracy: 0.886719
Epoch 10, Batch 253 -Loss:   349.1876 Validation Accuracy: 0.886719
Epoch 10, Batch 254 -Loss:   248.8911 Validation Accuracy: 0.886719
Epoch 10, Batch 255 -Loss:   255.5254 Validation Accuracy: 0.886719
Epoch 10, Batch 256 -Loss:   150.7892 Validation Accuracy: 0.890625
Epoch 10, Batch 257 -Loss:   159.3516 Validation Accuracy: 0.890625
Epoch 10, Batch 258 -Loss:   269.1634 Validation Accuracy: 0.890625
Epoch 10, Batch 259 -Loss:   305.3571 Validation Accuracy: 0.894531
Epoch 10, Batch 260 -Loss:   264.5325 Validation Accuracy: 0.894531
Epoch 10, Batch 261 -Loss:   306.8696 Validation Accuracy: 0.886719
Epoch 10, Batch 262 -Loss:   497.9378 Validation Accuracy: 0.898438
Epoch 10, Batch 263 -Loss:   447.8134 Validation Accuracy: 0.894531
Epoch 10, Batch 264 -Loss:   434.7628 Validation Accuracy: 0.894531
Epoch 10, Batch 265 -Loss:   305.7273 Validation

Epoch 10, Batch 372 -Loss:   353.2981 Validation Accuracy: 0.886719
Epoch 10, Batch 373 -Loss:   216.1406 Validation Accuracy: 0.890625
Epoch 10, Batch 374 -Loss:   325.6853 Validation Accuracy: 0.890625
Epoch 10, Batch 375 -Loss:   277.6452 Validation Accuracy: 0.890625
Epoch 10, Batch 376 -Loss:   326.8598 Validation Accuracy: 0.886719
Epoch 10, Batch 377 -Loss:   151.1886 Validation Accuracy: 0.882812
Epoch 10, Batch 378 -Loss:   395.2679 Validation Accuracy: 0.882812
Epoch 10, Batch 379 -Loss:   152.5244 Validation Accuracy: 0.882812
Epoch 10, Batch 380 -Loss:   272.8062 Validation Accuracy: 0.890625
Epoch 10, Batch 381 -Loss:   352.9434 Validation Accuracy: 0.882812
Epoch 10, Batch 382 -Loss:   133.8451 Validation Accuracy: 0.890625
Epoch 10, Batch 383 -Loss:   174.4736 Validation Accuracy: 0.878906
Epoch 10, Batch 384 -Loss:   181.9060 Validation Accuracy: 0.886719
Epoch 10, Batch 385 -Loss:   383.8945 Validation Accuracy: 0.890625
Epoch 10, Batch 386 -Loss:   307.3151 Validation